Option chains
=======

In [27]:
from ib_insync import *
util.startLoop()
import random
ib = IB()
if(ib.isConnected()):
    ib.disconnect()
random_id = random.randint(0, 9999)
ib.connect('127.0.0.1', 7496, clientId=random_id)

<IB connected to 127.0.0.1:7496 clientId=3611>

Suppose we want to find the options on the SPX, with the following conditions:

* Use the next three monthly expiries;
* Use strike prices within +- 20 dollar of the current SPX value;
* Use strike prices that are a multitude of 5 dollar.

To get the current market value, first create a contract for the underlyer (the S&P 500 index):

In [28]:
spx = Index('SPX', 'CBOE')
ib.qualifyContracts(spx)

[Index(conId=416904, symbol='SPX', exchange='CBOE', currency='USD', localSymbol='SPX')]

To avoid issues with market data permissions, we'll use delayed data:

In [29]:
ib.reqMarketDataType(4)

Then get the ticker. Requesting a ticker can take up to 11 seconds.

In [30]:
[ticker] = ib.reqTickers(spx)
ticker

Ticker(contract=Index(conId=416904, symbol='SPX', exchange='CBOE', currency='USD', localSymbol='SPX'), time=datetime.datetime(2022, 11, 14, 18, 7, 30, 79705, tzinfo=datetime.timezone.utc), marketDataType=3, minTick=0.01, bid=-1.0, bidSize=0.0, ask=-1.0, askSize=0.0, last=3997.6, lastSize=1.0, high=3999.31, low=3966.39, close=3992.93)

Take the current market value of the ticker:

In [31]:
spxValue = ticker.marketPrice()
if(str(spxValue)=="nan"):
    spxValue=3956
elif((spxValue)==0):
    spxValue=ticker.close
spxValue

3997.6

The following request fetches a list of option chains:

In [32]:
chains = ib.reqSecDefOptParams(spx.symbol, '', spx.secType, spx.conId)

util.df(chains)

,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes
0,SMART,416904,SPX,100,"[20221117, 20221215, 20230119, 20230216, 20230...","[100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700..."
1,CBOE,416904,SPXW,100,"[20221114, 20221115, 20221116, 20221117, 20221...","[100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700..."
2,SMART,416904,SPXW,100,"[20221114, 20221115, 20221116, 20221117, 20221...","[100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700..."
3,CBOE,416904,SPX,100,"[20221117, 20221215, 20230119, 20230216, 20230...","[100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700..."


These are four option chains that differ in ``exchange`` and ``tradingClass``. The latter is 'SPX' for the monthly and  'SPXW' for the weekly options. Note that the weekly expiries are disjoint from the monthly ones, so when interested in the weekly options the monthly options can be added as well.

In this case we're only interested in the monthly options trading on SMART:

In [33]:
chain = next(c for c in chains if c.tradingClass == 'SPX' and c.exchange == 'SMART')
chain

OptionChain(exchange='SMART', underlyingConId='416904', tradingClass='SPX', multiplier='100', expirations=['20221117', '20221215', '20230119', '20230216', '20230316', '20230420', '20230518', '20230615', '20230720', '20230817', '20230914', '20231019', '20231116', '20231214', '20240118', '20240620', '20241219', '20251218', '20261217', '20271216'], strikes=[100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700.0, 800.0, 900.0, 1000.0, 1100.0, 1150.0, 1200.0, 1250.0, 1300.0, 1350.0, 1375.0, 1400.0, 1450.0, 1475.0, 1500.0, 1525.0, 1550.0, 1575.0, 1600.0, 1625.0, 1650.0, 1675.0, 1700.0, 1725.0, 1750.0, 1775.0, 1800.0, 1825.0, 1850.0, 1875.0, 1900.0, 1925.0, 1950.0, 1975.0, 2000.0, 2025.0, 2050.0, 2075.0, 2100.0, 2125.0, 2150.0, 2175.0, 2200.0, 2225.0, 2250.0, 2275.0, 2300.0, 2325.0, 2350.0, 2375.0, 2400.0, 2425.0, 2450.0, 2475.0, 2500.0, 2525.0, 2550.0, 2575.0, 2600.0, 2625.0, 2650.0, 2675.0, 2700.0, 2725.0, 2750.0, 2775.0, 2800.0, 2825.0, 2850.0, 2870.0, 2875.0, 2880.0, 2890.0, 2900.0, 2910.0, 2920

What we have here is the full matrix of expirations x strikes. From this we can build all the option contracts that meet our conditions:

In [34]:
strikes = [strike for strike in chain.strikes
        if strike % 10 == 0
        and spxValue - 20 < strike < spxValue + 20
        ]

expirations = sorted(exp for exp in chain.expirations)[0]
rights = ['C']

contracts = [
        Option('SPX', expiration, strike, right, 'SMART', tradingClass='SPX')
        for right in rights
        for expiration in expirations
        for strike in strikes
        ]
print(strikes[0],strikes[-1])
len(contracts)

3980.0 4010.0


32

In [35]:

contracts = ib.qualifyContracts(*contracts)

Ambiguous contract: Option(symbol='SPX', lastTradeDateOrContractMonth='2', strike=3980.0, right='C', exchange='SMART', tradingClass='SPX'), possibles are [Contract(secType='OPT', conId=564055201, symbol='SPX', lastTradeDateOrContractMonth='20221117', strike=3980.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   221118C03980000', tradingClass='SPX'), Contract(secType='OPT', conId=569514412, symbol='SPX', lastTradeDateOrContractMonth='20221215', strike=3980.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   221216C03980000', tradingClass='SPX'), Contract(secType='OPT', conId=575215002, symbol='SPX', lastTradeDateOrContractMonth='20230119', strike=3980.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   230120C03980000', tradingClass='SPX'), Contract(secType='OPT', conId=581280618, symbol='SPX', lastTradeDateOrContractMonth='20230216', strike=3980.0, right='C', multiplier='100', exchange=

In [39]:
tickers = ib.reqTickers(*contracts)



In [42]:
len(tickers)


0

Now to get the market data for all options in one go:

In [37]:
while True:
    try:
        if(tickers[0].modelGreeks.delta!="nan"):
            break
    except:
        ib.sleep(15)
        print(tickers[0])
        


IndexError: list index out of range

In [ ]:
tickers[1]

IndexError: list index out of range

In [ ]:
d=[tick.modelGreeks.delta for tick in tickers]
print(d)

[-0.3820113914183043, -0.4028697848242917, -0.42443113293032986, -0.44630653581568747, -0.4683912358641398, -0.49076774197987005, -0.5132701737503088, -0.5357823103464316, -0.4374774205597405, -0.4448286418807249, -0.4522171440571304, -0.459658812720667, -0.4671442519271465, -0.4746663815255776, -0.48222593612406284, -0.48981553020444046, 0.618339582140387, 0.5971302630869213, 0.5754260447530566, 0.5536935008740775, 0.5316087828191498, 0.5092293564992099, 0.486766023733564, 0.4642228945650104, 0.5624891431936443, 0.5551713581192751, 0.5477828559428696, 0.540330742171642, 0.5328557480728535, 0.5253336184744224, 0.5177740638759372, 0.5101844697955595]


In [ ]:
ib.sleep(1)
a=tickers[0].marketPrice()
ib.sleep(1)
a


nan

In [ ]:
[ticker] = ib.reqTickers(contracts[0])
print(ticker)
b=ticker.marketPrice()
b

Error 10197, reqId 70: No market data during competing live session, contract: Option(conId=564055486, symbol='SPX', lastTradeDateOrContractMonth='20221117', strike=3960.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   221118P03960000', tradingClass='SPX')


Ticker(contract=Option(conId=564055486, symbol='SPX', lastTradeDateOrContractMonth='20221117', strike=3960.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   221118P03960000', tradingClass='SPX'), time=datetime.datetime(2022, 11, 14, 16, 44, 3, 397015, tzinfo=datetime.timezone.utc), marketDataType=3, minTick=0.0, bidGreeks=OptionComputation(tickAttrib=0, impliedVol=0.2100173122815329, delta=None, optPrice=None, pvDividend=3.1216504998015226, gamma=None, vega=None, theta=None, undPrice=3987.919921875), askGreeks=OptionComputation(tickAttrib=0, impliedVol=0.21230776183904648, delta=None, optPrice=None, pvDividend=3.1216504998015226, gamma=None, vega=None, theta=None, undPrice=3987.919921875), lastGreeks=OptionComputation(tickAttrib=0, impliedVol=0.20694681105468843, delta=None, optPrice=None, pvDividend=3.1216504998015226, gamma=None, vega=None, theta=None, undPrice=3987.919921875), modelGreeks=OptionComputation(tickAttrib=0, impliedVol=0.211246439018687

nan

In [ ]:
contracts[0]

Option(conId=564055486, symbol='SPX', lastTradeDateOrContractMonth='20221117', strike=3960.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   221118P03960000', tradingClass='SPX')

The option greeks are available from the ``modelGreeks`` attribute, and if there is a bid, ask resp. last price available also from ``bidGreeks``, ``askGreeks`` and ``lastGreeks``. For streaming ticks the greek values will be kept up to date to the current market situation.

In [ ]:
d=[tick.modelGreeks.delta for tick in tickers]
print(d)
#ib.disconnect()

[-0.3820113914183043, -0.4028697848242917, -0.42443113293032986, -0.44630653581568747, -0.4683912358641398, -0.49076774197987005, -0.5132701737503088, -0.5357823103464316, -0.4374774205597405, -0.4448286418807249, -0.4522171440571304, -0.459658812720667, -0.4671442519271465, -0.4746663815255776, -0.48222593612406284, -0.48981553020444046, 0.618339582140387, 0.5971302630869213, 0.5754260447530566, 0.5536935008740775, 0.5316087828191498, 0.5092293564992099, 0.486766023733564, 0.4642228945650104, 0.5624891431936443, 0.5551713581192751, 0.5477828559428696, 0.540330742171642, 0.5328557480728535, 0.5253336184744224, 0.5177740638759372, 0.5101844697955595]
